In [3]:
!pip install pyelftools --break-system-packages

Defaulting to user installation because normal site-packages is not writeable


In [1]:
%reload_ext autoreload
%autoreload now

from utils import system, BPFTOOL_PATH, dump_btf, normalize_btf, read_jsonl, gen_min_btf, Kind

In [6]:
from pathlib import Path

obj_dir = Path("..").resolve() / "bcc" / "libbpf-tools" / ".output"


def get_structs(obj_file):
    btf_file = gen_min_btf(obj_file)
    dump_btf(btf_file)
    jsonl_path = normalize_btf(btf_file)
    types = read_jsonl(jsonl_path)

    def filter_type(t):
        if t["kind"] not in [Kind.STRUCT]:  # Kind.UNION
            return False

        if t["name"] in ["(anon)", "user_pt_regs"]:
            return False

        return True

    return [t["name"] for t in types if filter_type(t)]

In [12]:
def normalize_hook_name(name):
    for hook_type, prefixes in [
        ("kprobe", ["kprobe/", "kretprobe/", "fentry/", "fexit/"]),
        ("tracepoint", ["tp_btf/", "raw_tp/", "tracepoint/"]),
        ("uprobe", ["uprobe/", "uretprobe/", "uprobe", "uretprobe"]),
        ("usdt", ["usdt"]),
        ("perf_event", ["perf_event"]),
    ]:
        for prefix in prefixes:
            if not name.startswith(prefix):
                continue

            name = name[len(prefix):]
            if "/" in name:
                assert prefix == "tracepoint/"
                name = name.rsplit("/", 1)[-1]
            return (hook_type, name)

    raise ValueError(f"Unknown hook type: {name}")


def get_hooks(obj_file):
    from elftools.elf.elffile import ELFFile

    with open(obj_file, "rb") as f:
        elf = ELFFile(f)

        hooks = set()
        for section in elf.iter_sections():
            if section.name.startswith("."):
                continue
            if section.name == "license":
                continue
            if section.header.sh_type == "SHT_PROGBITS":
                name = normalize_hook_name(section.name)
                hooks.add(name)
        return hooks


result = {}

for obj_file in sorted(obj_dir.glob("*.bpf.o")):
    hooks = get_hooks(obj_file)
    structs = get_structs(obj_file)
    result[obj_file.name] = (hooks, structs)


for name, (hooks, structs) in result.items():
    print(name)
    print("  hooks:", hooks)
    print("  structs:", structs)

/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.btf already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.h already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.txt already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.json already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bashreadline.bpf.jsonl already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.btf already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.h already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.txt already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.json already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/bindsnoop.bpf.jsonl already exists
/Users/szhong/Downloads/bpf-study/bcc/libbpf-tools/.output/biol